# 对齐检索和 LLM

为了使检索器与 LLM 更好地协同工作，还可以通过调整检索器的输出，将检索到的文档片段与 LLM 的输入进行对齐，使其更符合 LLM 的偏好，从而能够更好地理解和利用这些文档片段生成准确的回答。

[相关研究者](https://arxiv.org/abs/2312.10997)总结了两种将检索器输出与 LLM 偏好对齐的方法：

（1）增加 Adapter 插件。即在检索器和 LLM 生成器之间增加一个适配器，用于优化输入到 LLM 的上下文信息。

* [PRCA](https://arxiv.org/abs/2310.18347) 通过上下文提取阶段和奖励驱动阶段来训练适配器，然后使用基于 token 的自回归策略优化检索器的输出。
* [RECOMP](https://arxiv.org/abs/2310.04408)引入了用于生成摘要的提取式和生成式压缩器。这些压缩器要么选择相关句子，要么合成文档信息，创建针对多文档查询的定制摘要。
* [PKG](https://arxiv.org/abs/2305.04757) 提出了通过指令微调将知识整合到白盒模型中的创新方法。在这种方法中，检索器模块被直接替换以根据查询生成相关文档，从而解决微调过程中的困难并提升模型性能。

（2）微调检索器。通过引入大语言模型的反馈信号来优化检索器，并使用监督信号进行训练。
* [REPLUG](https://arxiv.org/abs/2301.12652) 计算检索到的文档的概率分布，然后通过计算 KL 散度进行监督训练。这种简单有效的训练方法利用 LLM 作为监督信号提高检索模型的表现，无需特定的交叉注意力机制。
* [AAR（增强适配检索器）](https://arxiv.org/abs/2305.17331) 提出一种通用适配器，可支持多下游任务适配。通过使用编码器-解码器架构，通过 FiD 交叉注意力分数识别 LM 偏好的文档，为预训练的检索器引入监督信号。随后，检索器通过硬负采样和标准交叉熵损失进行微调。最终，改良后的检索器可以直接应用于提高目标 LLM 在目标任务中的性能。还有研究表明 LLM 可能更倾向于关注可读性高的文档而非信息丰富的文档。
* [UPRISE](https://arxiv.org/abs/2303.08518) 训练了一个轻量级提示词检索器，能够根据零样本任务输入，从预构建的提示池中自动检索适配的提示词。UPRISE 同样使用固定的 LLM 微调提示检索器。LLM 和检索器都以提示-输入对作为输入，并利用 LLM 提供的分数指导检索器的训练，有效地将 LLM 视为数据集标注器。

下面分别介绍以 **PRCA** 为代表的 Adapter 插件方法，以及以 **REPLUG** 为代表的微调检索器方法。

> 本节对于 Adapter 插件、微调检索器相关内容仅作简单理论介绍，具体实现细节请读者参考相关论文

## 1.1 增加 Adapter 插件（进阶）

### PRCA 概述

PRCA（可插拔奖励驱动上下文适配器）是一种面向检索增强语言模型的轻量级适配模块，旨在优化检索结果与生成器之间的信息交互。其核心思想是通过强化学习机制动态调整检索内容，无需修改生成器（如黑盒大语言模型）的内部参数，即可显著提升问答任务性能。PRCA 以“插件”形式灵活嵌入检索器与生成器之间，仅需少量训练即可适配不同任务场景。

![](./figures/PRCA-1.png)

### 核心原理

PRCA 的核心设计包含以下三部分：
1. 模块化架构：
* 以插件形式部署于检索器与生成器之间。

2. 强化学习驱动优化：
* 奖励函数：基于生成答案的质量设计奖励信号，指导 PRCA 优化方向。
* 策略梯度（Policy Gradient）：通过最大化期望奖励，训练 PRCA 动态筛选、重组或加权检索文档中的关键信息。

3. token 自回归策略：
* 在生成答案阶段，PRCA 根据当前生成状态动态调整上下文表示，实现细粒度控制。

PRCA 的训练流程基于一个两阶段训练策略：

![](./figures/PRCA-2.png)

1. 上下文提取阶段（Contextual Extraction Stage）
* 在这一阶段，PRCA 被训练为从输入文本中提取上下文信息。输入文本由查询和检索器检索到的 Top-K 文档组成。PRCA 的目标是生成一个上下文丰富的提取结果，用于后续的生成器回答。
* 训练过程中，PRCA 的参数 $\theta$ 通过最小化提取的上下文 $C_\text{extracted}$ 与真实上下文 $C_\text{truth}$ 之间的差异来优化。
$$\min_{\theta} L(\theta) = -\frac{1}{N} \sum_{i = 1}^{N} C_{\text{truth}}^{(i)} \log(f_{\text{PRCA}}(S_{\text{input}}^{(i)}; \theta))$$

2. 奖励驱动阶段（Reward-Driven Stage）
* 在这一阶段，PRCA 的目标是将提取的上下文 $C_\text{extracted}$ 与生成器对齐，确保提取的信息能够有效指导生成器生成准确的答案。
* 由于生成器是黑盒模型，PRCA 通过强化学习进行优化。生成器的输出与真实答案之间的差异被用作奖励信号，指导 PRCA 的参数更新。
* 通过计算生成的上下文与真实答案之间的 ROUGE-L 分数来评估生成器的奖励。

## 1.2 LLM 监督学习（进阶）

### REPLUG 概述

REPLUG 是一种通过**外部检索系统增强黑盒语言模型**性能的框架。其核心思想是通过检索与输入相关的文档，并将这些文档作为额外上下文输入语言模型，从而帮助语言模型更好地生成答案。与以往的检索增强模型不同，REPLUG 不需要访问语言模型的内部参数或进行微调，仅通过修改输入即可实现增强，适用于商业 API 等黑盒场景。

![](./figures/REPLUG-1.png)

### 实现原理
REPLUG 的实现主要包括以下几个关键步骤：
1. 文档检索（Document Retrieval）

* 给定输入上下文 x，REPLUG 使用一个检索模型从外部语料库 D 中检索与 x 相关的文档集合 D'。
* 检索模型基于双编码器架构，通过计算输入上下文和文档的嵌入向量之间的余弦相似度来确定相关性。
* 检索出的文档按相似度排序，选取前 k 个文档作为最相关的文档集合 D'。

2. 输入重构（Input Reformulation）

* 为了将检索到的文档整合到语言模型的输入中，REPLUG 采用了一种集成策略。具体来说，将每个检索到的文档 $d \in D'$ 分别附加到输入上下文 x 的前面，形成新的输入序列 $d \circ x$。
* 对每个新的输入序列 $d \circ x$，分别通过语言模型进行预测，并将这些预测结果的概率进行加权平均，得到最终的输出概率：
$$ p(y | x, D') = \sum_{d \in D'} p(y | d \circ x) \cdot \lambda(d, x) $$
其中，权重 $\lambda(d, x)$ 基于文档 d 和输入上下文 x 之间的相似度得分：
$$ \lambda(d, x) = \frac{e^{s(d,x)}}{\sum_{d \in D'} e^{s(d,x)}} $$

![](./figures/REPLUG-2.png)

3. REPLUG LSR（LM-Supervised Retrieval）

* REPLUG LSR 是一种训练方案，通过使用语言模型的输出作为监督信号来进一步优化检索模型。
* REPLUG LSR 的训练目标是使检索模型找到的文档能够降低语言模型的困惑度。训练过程包括以下步骤：
- **计算检索概率**：对于每个检索到的文档 d，计算其检索概率 $PR(d | x)$。
- **计算语言模型概率**：对于每个检索到的文档 d，计算其语言模型概率 $Q_\text{LM}(d | x, y)$。
- **损失函数**：通过最小化检索概率分布和语言模型概率分布之间的 KL 散度来更新检索模型的参数：
  $$ L = \frac{1}{|B|} \sum_{x \in B} KL(PR(d | x) \parallel Q_\text{LM}(d | x, y)) $$
- **异步更新数据索引**：在训练过程中，每隔 T 个训练步骤后，重新计算文档嵌入并更新检索索引，以确保检索模型的参数更新后仍能高效检索。

![](./figures/REPLUG-3.png)